In [1]:
import math

import numpy
import pandas

import ipywidgets as widgets
from IPython.display import display, clear_output

import gensim
from gensim.corpora import Dictionary
from gensim.models import TfidfModel
from gensim.models.doc2vec import Doc2Vec, TaggedDocument

import pythainlp

from sklearn.metrics.pairwise import cosine_similarity
from sklearn.cluster import DBSCAN
from sklearn.cluster import KMeans

from data_tokenizer import load_corpus

from model.upgrade_sdc import UpgradeSDC
from model.sdc import SDC

### Load Data

In [2]:
file_name = 'ผู้บริโภค - TescoLotus'

corpus, labels = load_corpus('../data/' + file_name + '.txt')

len_corpus = len(corpus)
print('Total documents', len_corpus)

clusters = list(set(labels))
print(len(clusters), 'clusters')

f = open('../data/tokenized/tokenized_' + file_name + '.txt')
tokenized_corpus = eval(f.read())
f.close()

Total documents 268
1 clusters


### Preprocess Corpus

#### Remove Words

In [3]:
dictionary = Dictionary(tokenized_corpus)
print('origin:', len(dictionary), 'words')

dictionary.filter_extremes(no_below=2, no_above=0.7, keep_n=len(dictionary))
print('filter frequent words:', len(dictionary), 'words')

letter_words = [id for id in range(len(dictionary)) if len(dictionary[id]) <= 1] 
dictionary.filter_tokens(bad_ids=letter_words)
print('filter letter words:', len(dictionary), 'words')

stopwords = pythainlp.corpus.stopwords.words('thai')
stopwords.extend(['นี้'])
dictionary.add_documents([stopwords])
stopwords = [dictionary.token2id[word] for word in stopwords]
dictionary.filter_tokens(bad_ids=stopwords)
print('filter stop words:', len(dictionary), 'words')

origin: 1449 words
filter frequent words: 605 words
filter letter words: 604 words
filter stop words: 402 words


In [4]:
idx_corpus = [dictionary.doc2idx(doc) for doc in tokenized_corpus]

temp_corpus = []
for doc in idx_corpus:
    temp_corpus.append([dictionary[id] for id in doc if id >= 0])
idx_corpus = temp_corpus

#### Dimension Reduction

In [5]:
average_doc_size = 0
for doc in idx_corpus:
    average_doc_size += len(doc)
average_doc_size /= len(idx_corpus)
average_doc_size = math.ceil(average_doc_size)

df = dictionary.dfs
filtered_corpus = []
for doc in idx_corpus:
    new_doc = [(word, df[dictionary.token2id[word]]) for word in doc]
    new_doc.sort(reverse=True, key=lambda x: x[1])
    new_doc = new_doc[:average_doc_size]
    filtered_corpus.append([word for word, df in new_doc])

#### Doc2Vec

In [6]:
tagged_corpus = [TaggedDocument(doc, [i]) for i, doc in enumerate(idx_corpus)]
model = Doc2Vec(tagged_corpus, vector_size=average_doc_size, window=4, min_count=2, epochs=100)
model.delete_temporary_training_data(keep_doctags_vectors=True, keep_inference=True)

paragraph_vectors = [model.infer_vector(doc) for doc in idx_corpus]
paragraph_vectors = pandas.DataFrame(paragraph_vectors, dtype=float)

### Clustering

In [7]:
def get_onehot(corpus, weight):
    dictionary = Dictionary(corpus)
#     dictionary.filter_extremes(no_below=2, no_above=1, keep_n=len(dictionary))

    bow_corpus = [dictionary.doc2bow(doc) for doc in corpus]
    if weight == 'normal':
        weight_corpus = bow_corpus
    elif weight == 'tfidf':
        tfidf = TfidfModel(bow_corpus, smartirs='ltc')
        weight_corpus = [tfidf[doc] for doc in bow_corpus]

    unique_words = [dictionary[id] for id in range(len(dictionary))]
    array = numpy.zeros((len(corpus), len(unique_words)), dtype=float)
    for i, doc in enumerate(weight_corpus):
        for id, score in doc:
            array[i, id] = score

        if weight == 'normal' and len(doc) != 0:
#             array[i] = numpy.divide(array[i], len(idx_corpus[i]))
            array[i] = numpy.divide(array[i], len(doc))
    
    return pandas.DataFrame(array, columns=unique_words, dtype=float)

In [8]:
def generate_result(predicted_labels, marks):
    result = pandas.DataFrame()
    result['comment'] = corpus
    result['tokenized_comment'] = filtered_corpus
    result['label'] = labels
    result['predicted_label'] = predicted_labels
    if marks:
        result['marks'] = marks
    else:
        result['marks'] = -1
    return result

In [9]:
def eval_cluster(onehot_corpus, result):
    label_count = numpy.unique(result['predicted_label'])
    num_cluster = label_count[-1] + 1

    clusters = [[] for i in range(num_cluster)]
    corpus_centroid = []
    for i, label in result['predicted_label'].iteritems():
        clusters[label].append(numpy.array(onehot_corpus.iloc[i]))
        corpus_centroid.append(numpy.array(onehot_corpus.iloc[i]))
    corpus_centroid = numpy.mean(corpus_centroid, axis=0).reshape(1, -1)   

#     print('\tIntra cluster sim\tInter cluster sim\tIntra / Inter')
    compactness = 0
    centroids = []
    for i in range(num_cluster):
        size = len(clusters[i])
        if size != 0:
            centroid = numpy.mean(clusters[i], axis=0)
            centroids.append(centroid)
            centroid = centroid.reshape(1, -1)
            similarities = cosine_similarity(centroid, clusters[i])
            compactness += numpy.sum(similarities)

#             intra = numpy.sum(similarities) / size
#             inter = cosine_similarity(centroid, corpus_centroid)[0][0]
#             print(i, end='\t')
#             print(intra, end='\t')
#             print(inter, end='\t')
#             print(intra / inter)
    return compactness, centroids

In [10]:
min_samples = 7
eps = 0.32
epoch = 15

onehot_corpus = get_onehot(idx_corpus, 'normal')
# onehot_corpus = get_onehot(filtered_corpus, 'tfidf')

max_compactness = 0
for i in range(epoch):
    model = UpgradeSDC()
    _tpredicted_labels, _tmarks = model.predict(onehot_corpus, min_samples, eps)
    
#     _tmarks = None

#     model = SDC()
#     _tpredicted_labels = model.predict(onehot_corpus, min_samples, eps)
    
#     model = DBSCAN(metric='cosine', eps=eps, min_samples=min_samples).fit(onehot_corpus)
#     _tpredicted_labels = model.labels_ + 1

#     model = KMeans(n_clusters=7).fit(onehot_corpus)
#     _tpredicted_labels = model.labels_
    
    _tresult = generate_result(_tpredicted_labels, _tmarks)
    compactness, _tcentroids = eval_cluster(onehot_corpus, _tresult)
    
    if compactness > max_compactness:
        max_compactness = compactness
        predicted_labels = _tpredicted_labels
        result = _tresult
        centroids = _tcentroids
        
print(max_compactness)
label_count = numpy.unique(result['predicted_label'], return_counts=True)
num_cluster = label_count[0][-1] + 1
print(label_count, '\n')

133.03737413062925
(array([ 0,  1,  2,  3,  4,  5,  6,  7,  8,  9, 10, 11, 12, 13, 14]), array([64, 24, 17, 13, 30,  8, 11, 24, 11, 12, 10, 15,  8, 11, 10])) 



In [11]:
sims = cosine_similarity(centroids)
new_labels = [i for i in range(num_cluster)]
print(new_labels)
for i, row in reversed(list(enumerate(sims))):
    for j, value in reversed(list(enumerate(row[:i + 1]))):
        if i != j and value >= eps - eps / 20:
            print(i, j, value)
            base = min(new_labels[i], new_labels[j])
            new_labels[j] = base
            new_labels = [base if label == new_labels[i] else label for label in new_labels]
print(new_labels)

grouped_labels = numpy.zeros(len_corpus)
for i, label in enumerate(predicted_labels):
    grouped_labels[i] = new_labels[label]
new_result = generate_result(grouped_labels, None)

[0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14]
13 11 0.4098436643560912
13 7 0.32902541972458327
13 6 0.35333313083777973
13 2 0.4429618200631101
11 7 0.40056213740733193
11 2 0.45545348801275565
11 1 0.41367126411361455
10 5 0.4730826646223039
7 2 0.4771667414687708
7 1 0.5301231904595685
2 1 0.719262046092703
[0, 1, 1, 3, 4, 5, 1, 1, 8, 9, 5, 1, 12, 1, 14]


### Result

In [12]:
class Widget:
    def __init__(self, result, column_name):
        self.result = result
        self.column_name = column_name
        
        label_count = numpy.unique(result['predicted_label'])
        self.widget = widgets.ToggleButtons(
            options=[int(num) for num in label_count],
            disabled=False,
            button_style='',
        )
        
        self.widget.observe(self.on_click, names='index')
        self.on_click({'new' : 0})
        
    def on_click(self, change):
        clear_output()
        display(self.widget)
        new = self.widget.options[change['new']]
        for index, value in self.result[self.result['predicted_label'] == new].iterrows():
            if value['marks'] == 0:
                print("@", end="")
            elif value['marks'] == 1:
                print("*", end="")
            print(index, value[self.column_name])

In [13]:
# result.to_csv('../data/results/' + file_name + '_2.csv')

# result = pandas.read_csv('../data/results/' + file_name + '_2.csv')

# count = 0
# for index, value in result.iterrows():
#     if value['marks'] == -1:
#         count += 1
# print(count)

In [14]:
w1 = Widget(new_result, 'comment')

ToggleButtons(options=(0, 1, 3, 4, 5, 8, 9, 12, 14), value=0)

2 เลิกเปิดเพลง ข้าวแสนดี กับอีเครื่องกรองน้ำเพียว ได้แล้ว!!!!
8 อย่าบังคับน้องเข้าประชุมเชียร์ อย่าลงโทษโดยเหตุผลงี่เง่าๆ เกิดก่อนไม่กี่ปีเอง
14 เลือกผัก เลือกแล้ว เลือกอีก พอมาคิดเงินพนักงานยัดผักใส่ถุงจนผักหัก โอ้ยใจ ใจสลาย
26 ค.ว.ย ค่ะ พนักงานแม่งเกือบทุกคนเลย ไม่มีความคิด วิเคราะห์ แยกแยะไม่ได้ว่าเวลาลูกค้าซื้อผัก ซื้อหมูบด น้ำยาล้างจาน น้ำปลา หรืออะไรก็แล้วแต่ ถุงมึงมีใบใหญ่สุดแค่ไหนมึงเอามาใส่แม่งรวมกันหมดทุกอย่าง ผักกูเลือกดีๆเอาไม่ช้ำ อีดอกเอาไว้ล่างสุด เอาแพ็คหมูบดทับอีกเหี้ย ยัง‼️ยังไม่พอมึงเอาน้ำยาล้างจานน้ำปลาว่าทับแพ็คหมู สรุปกูถึงบ้าน ผักเละ แพ็คหมูแตก ติดอยู่กับถุงน้ำยาล้างจาน อีสัส เอาไว้ให้โครตเหง้ามึงกินมั้ยได้ของแบบนี้ แยกไม่เป็น สมองคิดไม่ออกว่ามันต้องแยก🖕🏻🖕🏻🖕🏻
29 โลตลาด พนง.ปากตลาด นินทาลค.เผาขน ตะโกนโหวกเหวกข้ามหัวลค. วันหวยออกสนั่นเป็นพิเศษ เห็นพฤติกรรมแล้วแย่ ปรับปรุงภาพลักษณ์เถอะ
30 ไม่อยากจะเม้าความสาระพัดของโลตัส ในฐานะที่เคยเป็นพนง.มาก่อน อิอิ
32 ไม่ต้องตัดแต้มเป็นคูปองเงินได้ไหมอ่ะ ไม่เคยได้ใช้เพราะ คูปองมันหายก่อน เอาแบบ พอไปซื้อแล้วแต้มครบค่อยถามว่าตัดแต้

In [15]:
w2 = Widget(result, 'comment')

ToggleButtons(options=(0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14), value=0)

2 เลิกเปิดเพลง ข้าวแสนดี กับอีเครื่องกรองน้ำเพียว ได้แล้ว!!!!
8 อย่าบังคับน้องเข้าประชุมเชียร์ อย่าลงโทษโดยเหตุผลงี่เง่าๆ เกิดก่อนไม่กี่ปีเอง
14 เลือกผัก เลือกแล้ว เลือกอีก พอมาคิดเงินพนักงานยัดผักใส่ถุงจนผักหัก โอ้ยใจ ใจสลาย
26 ค.ว.ย ค่ะ พนักงานแม่งเกือบทุกคนเลย ไม่มีความคิด วิเคราะห์ แยกแยะไม่ได้ว่าเวลาลูกค้าซื้อผัก ซื้อหมูบด น้ำยาล้างจาน น้ำปลา หรืออะไรก็แล้วแต่ ถุงมึงมีใบใหญ่สุดแค่ไหนมึงเอามาใส่แม่งรวมกันหมดทุกอย่าง ผักกูเลือกดีๆเอาไม่ช้ำ อีดอกเอาไว้ล่างสุด เอาแพ็คหมูบดทับอีกเหี้ย ยัง‼️ยังไม่พอมึงเอาน้ำยาล้างจานน้ำปลาว่าทับแพ็คหมู สรุปกูถึงบ้าน ผักเละ แพ็คหมูแตก ติดอยู่กับถุงน้ำยาล้างจาน อีสัส เอาไว้ให้โครตเหง้ามึงกินมั้ยได้ของแบบนี้ แยกไม่เป็น สมองคิดไม่ออกว่ามันต้องแยก🖕🏻🖕🏻🖕🏻
29 โลตลาด พนง.ปากตลาด นินทาลค.เผาขน ตะโกนโหวกเหวกข้ามหัวลค. วันหวยออกสนั่นเป็นพิเศษ เห็นพฤติกรรมแล้วแย่ ปรับปรุงภาพลักษณ์เถอะ
30 ไม่อยากจะเม้าความสาระพัดของโลตัส ในฐานะที่เคยเป็นพนง.มาก่อน อิอิ
32 ไม่ต้องตัดแต้มเป็นคูปองเงินได้ไหมอ่ะ ไม่เคยได้ใช้เพราะ คูปองมันหายก่อน เอาแบบ พอไปซื้อแล้วแต้มครบค่อยถามว่าตัดแต้

In [16]:
w3 = Widget(result, 'tokenized_comment')

ToggleButtons(options=(0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14), value=0)

2 ['ดี', 'เลิก', 'เครื่อง', 'น้ำ', 'ข้าว', 'อี']
8 ['อย่า', 'อย่า', 'กี่', 'ปี', 'บังคับ', 'น้อง', 'เชียร์', 'งี่', 'เง่า']
14 ['พนักงาน', 'เงิน', 'ถุง', 'ใส่', 'ผัก', 'ผัก', 'ผัก', 'เลือก', 'เลือก', 'เลือก', 'ยัด']
26 ['พนักงาน', 'ลูกค้า', 'คน', 'ถุง', 'ถุง', 'ซื้อ', 'ซื้อ', 'เวลา', 'ดี', 'ใส่', 'ติด']
29 ['ปรับปรุง', 'พนง', 'แย่', 'ตะโกน', 'ข้าม', 'หวย', 'พิเศษ', 'พฤติกรรม']
30 ['พนง', 'ฐานะ']
32 ['เงิน', 'ซื้อ', 'ดี', 'ถาม', 'อ่ะ', 'หาย', 'เล่น', 'ไหม', 'ไหม', 'คู', 'ปอง']
35 ['ลูกค้า', 'จ่าย', 'พนง', 'พนง', 'ติด', 'ใบ', 'ใบ', 'ใบ', 'ดู', 'ดู', 'นึง']
37 ['สาขา', 'ทำ', 'ทำ', 'ทำ', 'ปรับปรุง', 'เรื่อง', 'ถาม', 'ถาม', 'นึง', 'นึง', 'แม่ง']
43 ['สาขา', 'แถม', 'กิน', 'สด', 'แม่ง', 'แผนก', 'เหม็น', 'เสื้อผ้า']
45 ['เวลา', 'เวลา', 'บริการ', 'สินค้า', 'มารยาท', 'ถาม', 'เจอ', 'ตอน', 'บ้าน', 'หรอก', 'สีหน้า']
56 ['สาขา', 'สาขา', 'ขาย', 'อ่ะ', 'วาง', 'เน่า', 'เน่า', 'เน่า', 'คุณภาพ', 'อายุ', 'งี้']
60 ['รู้', 'ผัก', 'เน่า', 'เน่า', 'ทิ้ง', 'ป้ายเหลือง']
69 ['อย่า', 'ผัก', 'หมู', 'ทิ้ง', 'บด',

In [17]:
seed = 85
compare = 171

a = numpy.array(onehot_corpus.iloc[seed]).reshape(1, -1)
b = numpy.array(onehot_corpus.iloc[compare]).reshape(1, -1)
print(cosine_similarity(a,b))
print(filtered_corpus[seed])
print(filtered_corpus[compare])

# print(sims[29])

[[0.]]
['พนักงาน', 'สาขา', 'ลูกค้า', 'หน้า', 'ทำ', 'ทำ', 'คน', 'คน', 'เหมือน', 'บริการ', 'แย่']
['รู้จัก']
